In [1]:
import json

with open('documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [2]:
from sentence_transformers import SentenceTransformer

/home/codespace/.python/current/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [3]:
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

In [4]:
v = model.encode('I just discovered the course. Can I still join?')

In [5]:
len(v)

384

In [7]:
v.dot(v)

1.0

In [9]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [10]:
from tqdm.auto import tqdm

In [11]:
for doc in tqdm(documents):
    question = doc['question']
    text = doc['text']
    qt = question + ' ' + text

    doc['question_vector'] = model.encode(question)
    doc['text_vector'] = model.encode(text)
    doc['question_text_vector'] = model.encode(qt)

100%|██████████| 948/948 [01:56<00:00,  8.11it/s]


In [12]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:21<00:00, 44.57it/s]


In [13]:
query = 'I just discovered the course. Can I still join it?'

In [14]:
v_q = model.encode(query)

In [15]:
v_q

array([ 1.76149234e-02, -4.11324874e-02, -1.86437108e-02,  3.38308746e-03,
        3.42942253e-02,  4.63595130e-02, -7.58891627e-02, -6.14615604e-02,
       -6.74188137e-02, -2.43898910e-02, -1.31267095e-02,  6.47083074e-02,
        2.51724236e-02,  6.61560968e-02,  5.74242100e-02, -5.76662309e-02,
       -1.01838619e-01, -3.80568802e-02,  8.85191094e-03, -2.22330331e-03,
       -1.45974591e-01,  4.60105762e-02, -3.27742398e-02,  4.28988673e-02,
        4.11894433e-02, -4.12427485e-02,  3.89968306e-02, -9.35932994e-03,
        1.78947728e-02, -3.18374299e-02, -2.50403886e-03,  9.68546048e-03,
        1.55836577e-02,  4.38631177e-02,  3.93182784e-03,  2.77109090e-02,
        2.76588299e-03, -5.23474589e-02,  6.36804523e-03,  1.12894475e-02,
       -1.66701302e-02, -4.24622819e-02,  4.98241838e-03,  3.54115590e-02,
        4.26929221e-02,  6.29352704e-02, -1.76142342e-02, -7.38580376e-02,
        3.98312099e-02,  8.52345526e-02,  1.04327770e-02, -1.14520557e-01,
       -1.59268230e-02, -

In [16]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [17]:
def question_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('question_vector', v_q, course)

In [18]:
import pandas as pd

In [19]:
df_ground_truth = pd.read_csv('ground-truth-data.csv')

In [20]:
ground_truth = df_ground_truth.to_dict(orient='records')

In [21]:
ground_truth[0]

{'question': 'When does the course begin?',
 'course': 'data-engineering-zoomcamp',
 'document': 'c02e79ef'}

In [23]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [24]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [25]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [26]:
evaluate(ground_truth, question_vector_knn)

100%|██████████| 5088/5088 [01:54<00:00, 44.52it/s]


{'hit_rate': 0.7022405660377359, 'mrr': 0.6054900419287217}

ES text only: 0.7395720769397017, 0.6032418413658963

In [28]:
def text_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('text_vector', v_q, course)

In [29]:
evaluate(ground_truth, text_vector_knn)

100%|██████████| 5088/5088 [01:46<00:00, 47.98it/s]


{'hit_rate': 0.7535377358490566, 'mrr': 0.6422431865828103}

In [30]:
def question_text_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('question_text_vector', v_q, course)

evaluate(ground_truth, question_text_vector_knn)

100%|██████████| 5088/5088 [01:45<00:00, 48.11it/s]


{'hit_rate': 0.8331367924528302, 'mrr': 0.7487355870020969}

In [31]:
def elastic_search_knn_combined(vector, course):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": [
                    {
                        "script_score": {
                            "query": {
                                "term": {
                                    "course": course
                                }
                            },
                            "script": {
                                "source": """
                                    cosineSimilarity(params.query_vector, 'question_vector') + 
                                    cosineSimilarity(params.query_vector, 'text_vector') + 
                                    cosineSimilarity(params.query_vector, 'question_text_vector') + 
                                    1
                                """,
                                "params": {
                                    "query_vector": vector
                                }
                            }
                        }
                    }
                ],
                "filter": {
                    "term": {
                        "course": course
                    }
                }
            }
        },
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [32]:
def vector_combined_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn_combined(v_q, course)

evaluate(ground_truth, vector_combined_knn)

100%|██████████| 5088/5088 [01:47<00:00, 47.18it/s]


{'hit_rate': 0.8197720125786163, 'mrr': 0.7308045073375267}